<a href="https://colab.research.google.com/github/Hussain1488/Fake-News-Detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torchtext kagglehub
# !pip uninstall -y torch torchtext
!pip install torch==2.3.0 torchtext==0.18.0


In [13]:
import kagglehub
import torchtext
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from torchtext.data.utils import get_tokenizer
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from torch import nn




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")
print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/fake-and-real-news-dataset


# Preprocessing:

In [19]:

fake=pd.read_csv(path + '/Fake.csv')
real=pd.read_csv(path + '/True.csv')
fake['label'] = 0
real['label'] = 1


class preprocessing:

  def __init__(self, fake, real):
    self.fake = fake
    self.real = real

  def stop_word_removal(self):

    stop_words = set(stopwords.words('english'))

    for i, item in self.fake.iterrows():
      filtered_text = [token.lower() for token in item['text'].split() if token.lower() not in stop_words]
      filtered_title = [token.lower() for token in item['title'].split() if token.lower() not in stop_words]
      fake.at[i, 'text'] = filtered_text
      fake.at[i, 'title'] = filtered_title

    for i, item in self.real.iterrows():
      filtered_text = [token.lower() for token in item['text'].split() if token.lower() not in stop_words]
      filtered_title = [token.lower() for token in item['title'].split() if token.lower() not in stop_words]
      real.at[i, 'text'] = filtered_text
      real.at[i, 'title'] = filtered_title
      df = pd.concat([fake, real])
    return df

  def stemming(self, dataset):
    stemmer = PorterStemmer()
    dataset['text'] = dataset['text'].astype(object)
    dataset['title'] = dataset['title'].astype(object)

    dataset['text'] = dataset['text'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    dataset['title'] = dataset['title'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    return dataset

  def title_one_hot(self, dataset):
    one_hot = pd.get_dummies(dataset['subject'], prefix='subject', dtype=int)
    dataset = pd.concat([dataset, one_hot], axis=1)
    return dataset

  def main(self):
    df = self.stop_word_removal()
    df = self.stemming(df)
    df = self.title_one_hot(df)
    return df


preprocessing = preprocessing(fake, real)
df = preprocessing.main()



df

,title,text,subject,date,label,subject_Government News,subject_Middle-east,subject_News,subject_US_News,subject_left-news,subject_politics,subject_politicsNews,subject_worldnews
0,"[donald, trump, send, embarrass, new, year’, e...","[donald, trump, wish, american, happi, new, ye...",News,"December 31, 2017",0,0,0,1,0,0,0,0,0
1,"[drunk, brag, trump, staffer, start, russian, ...","[hous, intellig, committe, chairman, devin, nu...",News,"December 31, 2017",0,0,0,1,0,0,0,0,0
2,"[sheriff, david, clark, becom, internet, joke,...","[friday,, reveal, former, milwauke, sheriff, d...",News,"December 30, 2017",0,0,0,1,0,0,0,0,0
3,"[trump, obsess, even, obama’, name, code, webs...","[christma, day,, donald, trump, announc, would...",News,"December 29, 2017",0,0,0,1,0,0,0,0,0
4,"[pope, franci, call, donald, trump, christma, ...","[pope, franci, use, annual, christma, day, mes...",News,"December 25, 2017",0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21412,"['fulli, committed', nato, back, new, u.s., ap...","[brussel, (reuters), -, nato, alli, tuesday, w...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,0,1
21413,"[lexisnexi, withdrew, two, product, chines, ma...","[london, (reuters), -, lexisnexis,, provid, le...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,0,1
21414,"[minsk, cultur, hub, becom, author]","[minsk, (reuters), -, shadow, disus, soviet-er...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,0,1
21415,"[vatican, upbeat, possibl, pope, franci, visit...","[moscow, (reuters), -, vatican, secretari, sta...",worldnews,"August 22, 2017",1,0,0,0,0,0,0,0,1


In [20]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 44898 entries, 0 to 21416
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   title                    44898 non-null  object
 1   text                     44898 non-null  object
 2   subject                  44898 non-null  object
 3   date                     44898 non-null  object
 4   label                    44898 non-null  int64 
 5   subject_Government News  44898 non-null  int64 
 6   subject_Middle-east      44898 non-null  int64 
 7   subject_News             44898 non-null  int64 
 8   subject_US_News          44898 non-null  int64 
 9   subject_left-news        44898 non-null  int64 
 10  subject_politics         44898 non-null  int64 
 11  subject_politicsNews     44898 non-null  int64 
 12  subject_worldnews        44898 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 4.8+ MB
